In [2]:
from taucmdr.cf.storage.levels import USER_STORAGE
from taucmdr.error import ConfigurationError
from taucmdr.model.trial import Trial
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import nbformat
import plotly.graph_objs as pgo
from tau_profile_parser import TauProfileParser
import matplotlib
import pandas as pd

In [ ]:
def trial_barplot_figure(trial_id, metric):
    profile = TauProfileParser.parse('./mpi-profile-example')

    mdata = profile.interval_data()[[metric]]
    idata = mdata.unstack().sort_values((0, 0, 0), axis = 1, ascending = False).transpose().fillna(0)
    df = pd.concat([idata[:100], idata[:100].sum(level = 0).rename({metric: 'MISC'})], copy = False)
    indices = [str(i) for i in df.columns.values]

    data = [
        pgo.Bar(
            y = indices[::-1],
            x = r.values[::-1],
            name = idx[1],
            orientation = "h",
            hoverinfo = "text",
            text = [ "<br>".join([idx[1][i:i+75] for i in range(0, len(idx[1]), 75)])] * len(indices)
        ) for idx, r in df.iterrows()]

    data[-1]['marker'] = { 'color': '#333333' }

    layout = pgo.Layout(
        barmode = 'stack',
        showlegend = False,
        hovermode = 'closest',
        margin = pgo.layout.Margin(l = 0, r = 0, b = 0, t = 10))

    return pgo.Figure(data = data, layout = layout)

def trial_barplot_widget(trial_id, metric):
    return trial_barplot_figure(trial_id, metric)

trial_selector = widgets.Dropdown(options=[1,2,3], value=1)
metric_selector = widgets.Dropdown(options=["Exclusive", "Inclusive"], value="Exclusive")
interact(trial_barplot_widget, trial_id = trial_selector, metric = metric_selector)

In [ ]:
def profile_barplot_figure(trial_id, metric, indices):
    profile = TauProfileParser.parse('./mpi-profile-example')
    idata = profile.interval_data()
    
    ds = idata.loc[indices][[metric]].sort_values(metric)
    maxval = ds[metric].max()
    nrows = ds.shape[0]
    
    colors_str = "#1f77b4,#aec7e8,#ff7f0e,#ffbb78,#2ca02c,#98df8a,#d62728,#ff9896,#9467bd,#c5b0d5,#8c564b,#c49c94,#e377c2,#f7b6d2,#7f7f7f,#c7c7c7,#bcbd22,#dbdb8d,#17becf,#9edae5,#7fc97f,#beaed4,#fdc086,#ffff99,#386cb0,#f0027f,#bf5b17,#666666,#1b9e77,#d95f02,#7570b3,#e7298a,#66a61e,#e6ab02,#a6761d,#666666,#fbb4ae,#b3cde3,#ccebc5,#decbe4,#fed9a6,#ffffcc,#e5d8bd,#fddaec,#f2f2f2,#393b79,#5254a3,#6b6ecf,#9c9ede,#637939,#8ca252,#b5cf6b,#cedb9c,#8c6d31,#bd9e39,#e7ba52,#e7cb94,#843c39,#ad494a,#d6616b,#e7969c,#7b4173,#a55194,#ce6dbd,#de9ed6,#8dd3c7,#ffffb3,#bebada,#fb8072,#80b1d3,#fdb462,#b3de69,#fccde5,#d9d9d9,#bc80bd,#ccebc5,#ffed6f,#3182bd,#6baed6,#9ecae1,#c6dbef,#e6550d,#fd8d3c,#fdae6b,#fdd0a2,#31a354,#74c476,#a1d99b,#c7e9c0,#756bb1,#9e9ac8,#bcbddc,#dadaeb,#636363,#969696,#bdbdbd,#d9d9d9"
    colors = colors_str.split(',')

    data = [
        pgo.Bar(
            x = ds[metric],
            y = ds.index.values,
            orientation = 'h',
            marker = dict(color = colors[:nrows]),
            text = ds.index.values,
            textposition = 'outside'
        )]

    layout = pgo.Layout(
        autosize = True,
        height = 25 * nrows,
        margin = pgo.layout.Margin(l = 0, r = 0, b = 0, t = 10),
        xaxis = dict(range = (maxval * -0.1, maxval * 1.25)))

    return pgo.Figure(data = data, layout = layout)

def profile_barplot_widget(trial_id, metric, indices):
    return profile_barplot_figure(trial_id, metric, indices)

trial_selector = widgets.Dropdown(options = [1, 2, 3], value = 1)
metric_selector = widgets.Dropdown(options = ["Exclusive", "Inclusive"], value = "Exclusive")
indices_selector = widgets.Dropdown(options = [(0, 0, 0), (1, 0, 0)], value = (0, 0, 0))
interact(profile_barplot_widget, trial_id = trial_selector, metric = metric_selector, indices = indices_selector)

In [79]:
def runtime_breakdownplot_figure(trials, metric, topn):
    dfs = []
    for id in trials:
        profile = TauProfileParser.parse('.tau/mpi-examples/ring-sample/{}'.format(id))
        df = profile.interval_data()
        df.reset_index(inplace=True)
        df['Ranks'] = [df["Node"].nunique()] * df["Node"].shape[0]
        df["Timer"] = df["Timer"].apply(lambda x: x.strip())
        timer_total = df[metric].sum()
        df["Normed_Metric"] = df[metric].apply(lambda x: round(float(x) / timer_total, 6))
        df = df.groupby(["Ranks", "Timer"])["Normed_Metric"].sum().drop(columns=["index"]).reset_index()
        dfs.append(df)

    mdfs = pd.concat(dfs)

    rank_vals = mdfs["Ranks"].unique()
    rank_max = mdfs["Ranks"].max()
    topn_timers = mdfs[mdfs["Ranks"] == rank_max].sort_values("Normed_Metric", ascending=False)[:topn]["Timer"]

    mdfs = mdfs[mdfs["Timer"].isin(topn_timers)]

    for rank in rank_vals:
        normsum = mdfs[mdfs["Ranks"] == rank]["Normed_Metric"].sum()
        remsum = 1 - normsum
        mdfs = mdfs.append({"Ranks": rank, "Timer": "Misc", "Normed_Metric": remsum}, ignore_index=True)

    mdfs = mdfs.pivot(index='Timer', columns='Ranks', values='Normed_Metric')
    mdfs = mdfs.sort_values(rank_max, ascending=False)

    ranks = list(mdfs.columns)
    times = mdfs.values.tolist()
    timers = mdfs.index.values

    colors_str = "#1f77b4,#aec7e8,#ff7f0e,#ffbb78,#2ca02c,#98df8a,#d62728,#ff9896,#9467bd,#c5b0d5,#8c564b,#c49c94,#e377c2,#f7b6d2,#7f7f7f,#c7c7c7,#bcbd22,#dbdb8d,#17becf,#9edae5,#7fc97f,#beaed4,#fdc086,#ffff99,#386cb0,#f0027f,#bf5b17,#666666,#1b9e77,#d95f02,#7570b3,#e7298a,#66a61e,#e6ab02,#a6761d,#666666,#fbb4ae,#b3cde3,#ccebc5,#decbe4,#fed9a6,#ffffcc,#e5d8bd,#fddaec,#f2f2f2,#393b79,#5254a3,#6b6ecf,#9c9ede,#637939,#8ca252,#b5cf6b,#cedb9c,#8c6d31,#bd9e39,#e7ba52,#e7cb94,#843c39,#ad494a,#d6616b,#e7969c,#7b4173,#a55194,#ce6dbd,#de9ed6,#8dd3c7,#ffffb3,#bebada,#fb8072,#80b1d3,#fdb462,#b3de69,#fccde5,#d9d9d9,#bc80bd,#ccebc5,#ffed6f,#3182bd,#6baed6,#9ecae1,#c6dbef,#e6550d,#fd8d3c,#fdae6b,#fdd0a2,#31a354,#74c476,#a1d99b,#c7e9c0,#756bb1,#9e9ac8,#bcbddc,#dadaeb,#636363,#969696,#bdbdbd,#d9d9d9"
    colors = colors_str.split(',')

    data = []
    for i, time_vals in enumerate(times):
        data.append(pgo.Scatter(
            x = rank_vals,
            y = time_vals,
            mode = 'lines',
            line = dict(width = 0.5, color = colors[i]),
            stackgroup = 'one',
            name = timers[i][:75],
            text = timers[i][:75],
            hoverinfo = 'text',
            groupnorm = 'percent'
        ))

    layout = pgo.Layout(
        hovermode = 'closest',
        showlegend = False,
        xaxis_type = 'category',
        yaxis = dict(
            type = 'linear',
            range = [1, 100],
            ticksuffix='%'))

    fig = pgo.Figure(data = data, layout = layout)
    return fig

def runtime_breakdownplot_widget(trials, metric, topn):
    return runtime_breakdownplot_figure(trials, metric, topn)

trial_selector = widgets.SelectMultiple(options = [0, 1, 2, 3, 4], value = [2, 3, 4])
metric_selector = widgets.Dropdown(options = ["Exclusive", "Inclusive"], value = "Exclusive")
topn_selector = widgets.BoundedIntText(min = 1, max = 200, value = 10)
interact(runtime_breakdownplot_widget, trials = trial_selector, metric = metric_selector, topn = topn_selector)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KFNlbGVjdE11bHRpcGxlKGRlc2NyaXB0aW9uPXUndHJpYWxzJywgaW5kZXg9KDIsIDMsIDQpLCBvcHRpb25zPSgwLCAxLCAyLCAzLCA0KSwgdmFsdWU9KDLigKY=


<function __main__.runtime_breakdownplot_widget>